In [209]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [210]:
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")

In [211]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [212]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GEMINI_API_KEY)
url = [
    "https://langchain-ai.github.io/langgraph/"
]

In [213]:
docs = [WebBaseLoader(url).load()]
docs_list = [item for sublist in docs for item in sublist]
print(docs_list)

[Document(metadata={'source': 'https://langchain-ai.github.io/langgraph/', 'title': 'Home', 'description': 'Build language agents as graphs', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\n\n\nHome\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n          Skip to content\n        \n\n\n\n\n\n\n\nJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            \n              Home\n            \n          \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Initializing search\n          \n\n\n\n\n\n\n\n\n\n\n\n\n    GitHub\n  \n\n\n\n\n\n\n\n\n\n\n          \n  \n    \n  \n  Home\n\n        \n\n\n\n          \n  \n    \n  \n  API reference\n\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    GitHub\n  \n\n\n\n\n\n\n\n\n    \n  \n    Home\n  \n\n    \n  \n\n\n\n\n\n\n\n\n            \n  \n    Home\n  \n\n          \n\n\n\n\n\n    \n  \n    Get started\n  \n\n    \n 

In [214]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

In [215]:
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=embeddings,
)
retriever = vectorstore.as_retriever()

In [216]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(max_results=2)
# tools = [tool]
# tool.invoke("What's a 'node' in LangGraph?")


In [217]:
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict

from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

from pydantic import BaseModel, Field
from langchain_core.documents import Document

from langgraph.prebuilt import tools_condition
from langchain_groq import ChatGroq

# Generate Response Node
def generate(state):
    """
    Generate answer

    Args:
        state (messages): The current state

    Returns:
         dict: The updated state with re-phrased question
    """
    print("---GENERATE---")
    messages = state["messages"]
    question = messages[0].content
    last_message = messages[-1]

    docs = last_message.content

    # Prompt
    prompt = hub.pull("rlm/rag-prompt")

    # LLM
    llm =  ChatGroq(temperature=0, model_name="gemma2-9b-it", groq_api_key=GROQ_API_KEY)

    # Post-processing
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    # Chain
    rag_chain = prompt | llm | StrOutputParser()

    # Run
    response = rag_chain.invoke({"context": docs, "question": question})
    return {"messages": [response]}


print("*" * 20 + "Prompt[rlm/rag-prompt]" + "*" * 20)
prompt = hub.pull("rlm/rag-prompt",api_key= LANGSMITH_API_KEY).pretty_print()  # Show what the prompt looks like


********************Prompt[rlm/rag-prompt]********************
================================ Human Message =================================

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


In [218]:
# Web search Agent

def web_search_agent(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["messages"]
    query = question[0].content
    print(question)
    # Web search
    docs = web_search_tool.invoke({"query": query})
    web_results = "\n".join([d["content"] for d in docs])
    # web = Document(page_content=web_results)

    # We return a list, because this will get added to the existing list
    return {"messages": [web_results]}

In [219]:
# Reframe the question
def rewrite(state):
    """
    Transform the query to produce a better question.

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with re-phrased question
    """

    print("---TRANSFORM QUERY---")
    messages = state["messages"]
    question = messages[0].content

    msg = [
        HumanMessage(
            content=f""" \n
    Look at the input and try to reason about the underlying semantic intent / meaning. \n
    Here is the initial question:
    \n ------- \n
    {question}
    \n ------- \n
    Formulate an improved question: """,
        )
    ]

    llm =  ChatGroq(temperature=0, model_name="gemma2-9b-it", groq_api_key=GROQ_API_KEY)
    response = llm.invoke(msg)
    return {"messages": [response]}


In [220]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    name = "retrieve_blog_posts",
    description= "Search and return information about langgraph",
)


tools = [retriever_tool]

# retrieval agent
def retrieve_agent(state):
    """
    Invokes the agent model to generate a response based on the current state. Given
    the question, it will decide to retrieve using the retriever tool, or simply end.

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with the agent response appended to messages
    """
    print("---CALL AGENT---")
    messages = state["messages"]
    model =  ChatGroq(temperature=0, model_name="gemma2-9b-it", groq_api_key=GROQ_API_KEY)
    model = model.bind_tools(tools)
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [221]:
# Document grader
def grade_documents(state) -> Literal["generate_response", "web_search"]:
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (messages): The current state

    Returns:
        str: A decision for whether the documents are relevant or not
    """

    print("---CHECK RELEVANCE---")

    # Data model
    class grade(BaseModel):
        """Binary score for relevance check."""

        binary_score: str = Field(description="Relevance score 'yes' or 'no'")

    # LLM
    model =  ChatGroq(temperature=0, model_name="gemma2-9b-it", groq_api_key=GROQ_API_KEY)

    # LLM with tool and validation
    llm_with_tool = model.with_structured_output(grade)

    # Prompt
    prompt = PromptTemplate(
        template="""You are a grader assessing relevance of a retrieved document to a user question. \n
        Here is the retrieved document: \n\n {context} \n\n
        Here is the user question: {question} \n
        If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
        Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.""",
        input_variables=["context", "question"],
    )

    # Chain
    chain = prompt | llm_with_tool

    messages = state["messages"]
    last_message = messages[-1]

    question = messages[0].content
    docs = last_message.content

    scored_result = chain.invoke({"question": question, "context": docs})

    score = scored_result.binary_score

    if score == "yes":
        print("---DECISION: DOCS RELEVANT---")
        return "generate_response"

    else:
        print("---DECISION: DOCS NOT RELEVANT---")
        print(score)
        return "web_search"

## Define graph “workflow” add nodes and edges/relationships and compile

In [222]:
from typing import Annotated, TypedDict,Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    # The add_messages function defines how an update should be processed
    # Default is to replace. add_messages says "append"
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [223]:
from langgraph.graph import START,END,StateGraph
from langgraph.prebuilt import ToolNode


workflow = StateGraph(AgentState)


# Nodes
workflow.add_node("web_search",web_search_agent)
workflow.add_node("retrieve",retrieve_agent)

retriever_node = ToolNode([retriever_tool])  # create retriever tool node

workflow.add_node("retriever", retriever_node)
workflow.add_node("generate_response", generate)

# Edges
workflow.add_edge(START,"retrieve")
workflow.add_edge("web_search","generate_response")
workflow.add_conditional_edges("retrieve",
                               tools_condition,
                               {"tools":"retriever"},
                               END)
workflow.add_conditional_edges("retriever",grade_documents)
workflow.add_edge("generate_response",END)

# Compile workflow
graph = workflow.compile()



In [224]:
import pprint

inputs = {
    "messages": [
        ("user", "how do you play cricket?"),
    ]
}
for output in graph.stream(inputs):
    for key, value in output.items():
        pprint.pprint(f"Output from node '{key}':")
        pprint.pprint("---")
        pprint.pprint(value, indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")

---CALL AGENT---
"Output from node 'retrieve':"
'---'
{ 'messages': [ AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_p2f5', 'function': {'arguments': '{"query":"how to play cricket"}', 'name': 'retrieve_blog_posts'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 961, 'total_tokens': 1048, 'completion_time': 0.158181818, 'prompt_time': 0.034005434, 'queue_time': 0.017939072, 'total_time': 0.192187252}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e14c506b-5b85-4984-8a5b-ed753cd997ad-0', tool_calls=[{'name': 'retrieve_blog_posts', 'args': {'query': 'how to play cricket'}, 'id': 'call_p2f5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 961, 'output_tokens': 87, 'total_tokens': 1048})]}
'\n---\n'
---CHECK RELEVANCE---
---DECISION: DOCS NOT RELEVANT---
no
"Output from node 'retriever':"
'---'
{ 'messages': [ ToolMessage(conte

/Users/kunalkothavade/Work/Data Science/GenAI/.lang_venv/lib/python3.11/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


"Output from node 'generate_response':"
'---'
{ 'messages': [ 'Cricket is played with two teams of 11 players each. One team '
                'bats, trying to score runs by hitting a ball with a bat, '
                'while the other team bowls and fields the ball, aiming to get '
                'the batsmen out.  The team with the most runs at the end of '
                'the game wins. \n']}
'\n---\n'


In [226]:
import pprint

inputs = {
    "messages": [
        ("user", "what are tools in langgraph"),
    ]
}
for output in graph.stream(inputs):
    for key, value in output.items():
        pprint.pprint(f"Output from node '{key}':")
        pprint.pprint("---")
        pprint.pprint(value, indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")

---CALL AGENT---
"Output from node 'retrieve':"
'---'
{ 'messages': [ AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5ref', 'function': {'arguments': '{"query":"tools in langgraph"}', 'name': 'retrieve_blog_posts'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 961, 'total_tokens': 1048, 'completion_time': 0.158181818, 'prompt_time': 0.033900298, 'queue_time': 0.019200658999999995, 'total_time': 0.192082116}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-35b87934-e839-40ba-abc9-5ecb025ba129-0', tool_calls=[{'name': 'retrieve_blog_posts', 'args': {'query': 'tools in langgraph'}, 'id': 'call_5ref', 'type': 'tool_call'}], usage_metadata={'input_tokens': 961, 'output_tokens': 87, 'total_tokens': 1048})]}
'\n---\n'
---CHECK RELEVANCE---
---DECISION: DOCS RELEVANT---
"Output from node 'retriever':"
'---'
{ 'messages': [ ToolMessage(conte

/Users/kunalkothavade/Work/Data Science/GenAI/.lang_venv/lib/python3.11/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


"Output from node 'generate_response':"
'---'
{ 'messages': [ 'LangGraph uses tools defined with the `@tool` decorator.  \n'
                'The provided example defines a tool called `search` that '
                'returns weather information. \n'
                'Tools are then used within a `ToolNode` to enable the agent '
                'to interact with them. \n'
                '\n'
                '\n']}
'\n---\n'
